In [13]:
import cv2,os

data_path=r"C:\Users\Akash\dataset"
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)

{'mask': 0, 'without_mask': 1}
['mask', 'without_mask']
[0, 1]


In [17]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
    print(img_names)   
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image


['100100_faces.jpg', '100100_faces.jpg_0_128.jpg', '100100_faces.jpg_0_160.jpg', '100100_faces.jpg_0_2208.jpg', '100100_faces.jpg_0_2769.jpg', '100100_faces.jpg_0_347.jpg', '100100_faces.jpg_0_3502.jpg', '100100_faces.jpg_0_3571.jpg', '100100_faces.jpg_0_3773.jpg', '100100_faces.jpg_0_47.jpg', '100100_faces.jpg_0_4904.jpg', '100100_faces.jpg_0_4996.jpg', '100100_faces.jpg_0_5019.jpg', '100100_faces.jpg_0_5304.jpg', '100100_faces.jpg_0_5434.jpg', '100100_faces.jpg_0_5581.jpg', '100100_faces.jpg_0_6301.jpg', '100100_faces.jpg_0_6319.jpg', '100100_faces.jpg_0_6321.jpg', '100100_faces.jpg_0_6954.jpg', '100100_faces.jpg_0_7320.jpg', '100100_faces.jpg_0_7342.jpg', '100100_faces.jpg_0_7728.jpg', '100100_faces.jpg_0_8076.jpg', '100100_faces.jpg_0_813.jpg', '100100_faces.jpg_0_8209.jpg', '100100_faces.jpg_0_8652.jpg', '100100_faces.jpg_0_8939.jpg', '100100_faces.jpg_0_8977.jpg', '100100_faces.jpg_0_9094.jpg', '100100_faces.jpg_0_9500.jpg', '100100_faces.jpg_0_9880.jpg', '102102_faces.jpg', '102

Exception: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

['101101_faces.jpg', '101101_faces.jpg_0_128.jpg', '101101_faces.jpg_0_1541.jpg', '101101_faces.jpg_0_1706.jpg', '101101_faces.jpg_0_171.jpg', '101101_faces.jpg_0_1749.jpg', '101101_faces.jpg_0_2521.jpg', '101101_faces.jpg_0_3542.jpg', '101101_faces.jpg_0_4052.jpg', '101101_faces.jpg_0_4166.jpg', '101101_faces.jpg_0_4816.jpg', '101101_faces.jpg_0_4863.jpg', '101101_faces.jpg_0_4918.jpg', '101101_faces.jpg_0_5068.jpg', '101101_faces.jpg_0_5083.jpg', '101101_faces.jpg_0_5988.jpg', '101101_faces.jpg_0_6180.jpg', '101101_faces.jpg_0_634.jpg', '101101_faces.jpg_0_6392.jpg', '101101_faces.jpg_0_6438.jpg', '101101_faces.jpg_0_6882.jpg', '101101_faces.jpg_0_7126.jpg', '101101_faces.jpg_0_7134.jpg', '101101_faces.jpg_0_7165.jpg', '101101_faces.jpg_0_7505.jpg', '101101_faces.jpg_0_7575.jpg', '101101_faces.jpg_0_7786.jpg', '101101_fa

In [19]:
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)
np.save('data',data)
np.save('target',new_target)

In [21]:
import numpy as np

data=np.load('data.npy')
target=np.load('target.npy')

#loading the save numpy arrays in the previous code
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten,Dropout
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 4389 samples, validate on 1098 samples
Epoch 1/20
4389/4389 [==============================] - 185s 42ms/sample - loss: 0.6883 - acc: 0.5714 - val_loss: 0.6832 - val_acc: 0.5783
Epoch 2/20
4389/4389 [==============================] - 186s 42ms/sample - loss: 0.6847 - acc: 0.5726 - val_loss: 0.6813 - val_acc: 0.5783
Epoch 3/20
4389/4389 [==============================] - 185s 42ms/sample - loss: 0.6835 - acc: 0.5726 - val_loss: 0.6828 - val_acc: 0.5783
Epoch 4/20
4389/4389 [==============================] - 185s 42ms/sample - loss: 0.6836 - acc: 0.5726 - val_loss: 0.6809 - val_acc: 0.5783
Epoch 5/20
4389/4389 [==============================] - 186s 42ms/sample - loss: 0.6836 - acc: 0.5726 - val_loss: 0.6814 - val_acc: 0.5783
Epoch 6/20
4389/4389 [==============================] - 185s 42ms/sample - loss: 0.6828 - a